In [4]:
import advent
import numpy as np
advent.scrape(2024, 12, 'csv')
data = advent.get_char_grid(12)
padded_data = np.pad(data, 1, constant_values=['.'])

In [5]:
from advent.utils import tadd2

offsets = [(-1, 0), (1, 0), (0, 1), (0, -1)]

def get_score(leaves: set[tuple[int, int]]):
    # Score = area * perimeter
    area = len(leaves)
    perimeter = 0
    for leaf in leaves:
        for o in offsets:
            if tadd2(leaf, o) not in leaves: perimeter += 1
    return area * perimeter

def complete_field(flower: tuple[int, int], field: set[tuple[int, int]]):
    field.add(flower)
    for o in offsets:
        newflower = tadd2(flower, o)
        if newflower not in field and padded_data[newflower] == padded_data[flower]:
            field = complete_field(newflower, field)
    return field
    

fields = set([])
result = 0
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        field = complete_field((i+1, j+1), set([]))
        # This is kind of stupid brute force: we calculate the area for every (i, j)
        # even though 99% of the time its redundant but oh well it works
        if frozenset(field) not in fields:
            fields.add(frozenset(field))
            result += get_score(field)
print(result) # 40495393 too high

1489582


In [ ]:
# Part 2

# We only need to rewrite the 'get_score' function

# This is likely to be terrible... but what I decided to do for side calculations is
# a side is set[tuple[tuple[int, int], tuple[int, int]]] : set of pairs of coords
# the coords on either side of the side, where a is *inside* the shape
from advent.utils import tsub2, tadd2

flip = lambda x: (x[1], x[0])
flip2 = lambda x: (-x[1], -x[0])

def expand_side(a: tuple[int, int], b:tuple[int, int], side: set|None=None):
    # a must be inside the shape and b must be outside the shape!!
    # This function just recursively keeps traveling along the side
    # Until the side ends on both sides
    if side is None: side=set([])
    side.add((a, b))
    offset = tsub2(a, b)
    left = tadd2(a, flip(offset)), tadd2(b, flip(offset))
    right = tadd2(a, flip2(offset)), tadd2(b, flip2(offset))
    if padded_data[a] == padded_data[left[0]] and padded_data[a] != padded_data[left[1]] and left not in side:
        expand_side(left[0], left[1], side)
    if padded_data[a] == padded_data[right[0]] and padded_data[a] != padded_data[right[1]] and right not in side:
        expand_side(right[0], right[1], side)
    return side

def get_score2(field: set[tuple[int, int]]):
    sides: set[frozenset[tuple[tuple[int, int], tuple[int, int]]]] = set([])

    # For each element in the field, calculate all its sides and add them to sides
    # Obviously this will massively overcount sides, but because sides is a set
    # all duplicates will be naturally removed, ending us with a list of unique sides
    for a in field:
        for o in offsets:
            b = tadd2(a, o)
            if padded_data[a] != padded_data[b]:
                sides.add(frozenset(expand_side(a, b)))
    return len(field) * len(sides)

result = 0
for field in fields:
    result += get_score2(field)
print(result)

914966
